In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import Adam


In [3]:
# Define paths to dataset
base_dir = '/content/drive/MyDrive/FYP/chest_xray'  # Adjust the path if necessary
train_dir = os.path.join(base_dir, '/content/drive/MyDrive/FYP/chest_xray/train')
test_dir = os.path.join(base_dir, '/content/drive/MyDrive/FYP/chest_xray/test')


In [4]:
# Define the data generator
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)


In [5]:
# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images
    batch_size=32,
    class_mode='binary',  # For binary classification (Normal vs Pneumonia)
    subset='training'
)

Found 4187 images belonging to 2 classes.


In [6]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 1045 images belonging to 2 classes.


In [7]:
# Load test data
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 624 images belonging to 2 classes.


In [8]:
print("Training set class distribution:", train_generator.classes.sum())
print("Validation set class distribution:", validation_generator.classes.sum())
print("Test set class distribution:", test_generator.classes.sum())

Training set class distribution: 3107
Validation set class distribution: 776
Test set class distribution: 390


In [9]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,735,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10  # Adjust epochs based on your setup
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


131/131 ━━━━━━━━━━━━━━━━━━━━ 1408s 10s/step - accuracy: 0.7858 - loss: 0.4302 - val_accuracy: 0.9455 - val_loss: 0.1512
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - accuracy: 0.9522 - loss: 0.1198 - val_accuracy: 0.9541 - val_loss: 0.1210
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 260s 2s/step - accuracy: 0.9654 - loss: 0.0950 - val_accuracy: 0.9617 - val_loss: 0.1152
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 264s 2s/step - accuracy: 0.9713 - loss: 0.0905 - val_accuracy: 0.9569 - val_loss: 0.1352
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.9718 - loss: 0.0749 - val_accuracy: 0.9646 - val_loss: 0.1158
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.9830 - loss: 0.0444 - val_accuracy: 0.9579 - val_loss: 0.1406
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 338s 2s/step - accuracy: 0.9811 - loss: 0.0511 - val_accuracy: 0.9627 - val_loss: 0.1809
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 305s 2s/step - accuracy: 0.9799 - loss: 0.0546 - val_accuracy: 0.9

In [11]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 143s 7s/step - accuracy: 0.5081 - loss: 4.0972
Test Accuracy: 73.56%


In [12]:
# Generate predictions
predictions = (model.predict(test_generator) > 0.5).astype("int32")


20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 684ms/step


In [13]:
# Confusion Matrix and Classification Report
print("Confusion Matrix")
print(confusion_matrix(test_generator.classes, predictions))


Confusion Matrix
[[ 70 164]
 [  1 389]]


In [14]:
print("Classification Report")
print(classification_report(test_generator.classes, predictions, target_names=test_generator.class_indices.keys()))


Classification Report
              precision    recall  f1-score   support

      NORMAL       0.99      0.30      0.46       234
   PNEUMONIA       0.70      1.00      0.83       390

    accuracy                           0.74       624
   macro avg       0.84      0.65      0.64       624
weighted avg       0.81      0.74      0.69       624

